In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# The Brain Tumor Detection

In [2]:
import pandas as pd
import numpy as np
import re
import glob
from PIL import Image
from sklearn.metrics import classification_report

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [3]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 16098, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 16098 (delta 15), reused 14 (delta 2), pack-reused 16056
Receiving objects: 100% (16098/16098), 14.73 MiB | 6.31 MiB/s, done.
Resolving deltas: 100% (11047/11047), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.4/660.4 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.1.0 which is incompatible.


In [5]:
import torch
from yolov5 import utils

display = utils.notebook_init()

YOLOv5 🚀 v7.0-247-g3f02fde Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.0/78.2 GB disk)


In [5]:
# Train YOLOv5s
!python train.py --img 640 --batch 16 --epochs 50 --data /content/drive/MyDrive/01_DataScience/09_BRAIN_TUMOR_DETECTION/01_DATASET/yolo_v5/brain.yaml --weights yolov5s.pt --cache --project /content/drive/MyDrive/01_DataScience/09_BRAIN_TUMOR_DETECTION/02_RESULTS_YOLO/YOLOv5

2023-12-11 09:46:57.303867: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 09:46:57.303934: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 09:46:57.303979: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/01_DataScience/09_BRAIN_TUMOR_DETECTION/01_DATASET/yolo_v5/brain.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_

### Loading weight for models

In [6]:
model_v5s = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/01_DataScience/09_BRAIN_TUMOR_DETECTION/02_RESULTS_YOLO/YOLOv5/exp3/weights/best.pt')

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 v7.0-247-g3f02fde Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [12]:
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [18]:
def prediction(path_to_val_img):
  img_val = []
  path_to_img = path_to_val_img
  for img in sorted(glob.glob(f'{path_to_img}'), key=numericalSort):
    n= Image.open(img)
    i= np.asarray(n)
    img_val.append(i)

  predict = model_v5s(img_val)

  return predict

In [28]:
predict=prediction("/content/drive/MyDrive/01_DataScience/09_BRAIN_TUMOR_DETECTION/01_DATASET/yolo_v5/images/val/*")

In [ ]:
len(predict)

30

In [ ]:
print(predict)

image 1/30: 630x630 (no detections)
image 2/30: 630x630 (no detections)
image 3/30: 225x225 (no detections)
image 4/30: 250x201 (no detections)
image 5/30: 225x225 (no detections)
image 6/30: 192x192 (no detections)
image 7/30: 417x428 (no detections)
image 8/30: 251x201 (no detections)
image 9/30: 201x173 (no detections)
image 10/30: 201x173 (no detections)
image 11/30: 168x300 (no detections)
image 12/30: 183x275 (no detections)
image 13/30: 168x300 (no detections)
image 14/30: 197x177 (no detections)
image 15/30: 217x232 1 tumor
image 16/30: 380x310 1 tumor
image 17/30: 347x300 1 tumor
image 18/30: 173x189 1 tumor
image 19/30: 342x273 2 tumors
image 20/30: 256x256 (no detections)
image 21/30: 344x279 (no detections)
image 22/30: 331x272 1 tumor
image 23/30: 331x260 1 tumor
image 24/30: 236x213 2 tumors
image 25/30: 256x197 1 tumor
image 26/30: 251x201 1 tumor
image 27/30: 295x283 1 tumor
image 28/30: 352x281 2 tumors
image 29/30: 960x781 1 tumor
image 30/30: 349x292 1 tumor
Speed: 2

In [ ]:
#predict.save()

In [ ]:
predict.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def labels(path_to_val_ann):
  pred_lab=[]
  for x in range(len(predict)):
    aa=predict.pandas().xyxy[x]
    if len(aa)==0:
      pred_lab.append(0)
    else:
      pred_lab.append(1)

  txt_img = []
  for x in sorted(glob.glob(f"{path_to_val_ann}"), key=numericalSort):
    lab=x.split('/')[-1]
    txt_img.append(lab.split('.')[0])

  true_lab=[]
  for x in txt_img:
    if 'Y' in x:
      true_lab.append(1)
    else:
      true_lab.append(0)

  return pred_lab, true_lab

In [ ]:
pred_lab, true_lab = labels("/content/drive/MyDrive/01_DataScience/09_BRAIN_TUMOR_DETECTION/01_DATASET/yolo_v5/labels/val/*.txt")

In [ ]:
print(true_lab)
print(pred_lab)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
print(classification_report(true_lab, pred_lab))

              precision    recall  f1-score   support

           0       0.88      0.93      0.90        15
           1       0.93      0.87      0.90        15

    accuracy                           0.90        30
   macro avg       0.90      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30

